In [46]:
#연관분석 #장바구니 분석 #패턴들을 찾아서 분석하는것 
import mlxtend  
from mlxtend.frequent_patterns import apriori  
from mlxtend.frequent_patterns import association_rules  

dataset = [['Milk', 'Cookie', 'Apple', 'Beans', 'Eggs', 'Yogurt'],
          ['Coke', 'Cookie', 'Apple', 'Beans', 'Eggs', 'Yogurt'],
          ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
          ['Milk', 'Orange', 'Corn', 'Beans', 'Yogurt'],
          ['Corn', 'Cookie', 'Cookie', 'Beans', 'Ice cream', 'Eggs']]  
type(dataset)

list

In [47]:
# apriori 알고리즘은 list 글자형태가 아니라 True and False 형태로 해야함,
import pandas as pd  
from mlxtend.preprocessing import TransactionEncoder  
te = TransactionEncoder()  # 트랙잭션 생성을 위해 Encoder 객체 생성  
#해당 객체에 dataset 자료를 적합시키고 트랙잭션 형태로 변환  
te_ary = te.fit(dataset).transform(dataset)  
te_ary  

array([[ True,  True, False,  True, False,  True, False, False,  True,
        False,  True],
       [ True,  True,  True,  True, False,  True, False, False, False,
        False,  True],
       [ True, False, False, False, False,  True, False,  True,  True,
        False, False],
       [False,  True, False, False,  True, False, False, False,  True,
         True,  True],
       [False,  True, False,  True,  True,  True,  True, False, False,
        False, False]])

In [48]:
df = pd.DataFrame(te_ary, columns = te.columns_)  
df

,Apple,Beans,Coke,Cookie,Corn,Eggs,Ice cream,Kidney Beans,Milk,Orange,Yogurt
0,True,True,False,True,False,True,False,False,True,False,True
1,True,True,True,True,False,True,False,False,False,False,True
2,True,False,False,False,False,True,False,True,True,False,False
3,False,True,False,False,True,False,False,False,True,True,True
4,False,True,False,True,True,True,True,False,False,False,False


In [49]:
# mlxtend Apriori 알고리즘 적용하기  
import mlxtend  
from mlxtend.frequent_patterns import apriori  
from mlxtend.frequent_patterns import association_rules  
apriori(df, min_support = 0.6)  # 최소 지지도를 0.6으로 하여 Apriori에 적용하기  
#전제 구매내역에서 60%임을 의미한다. 
apriori(df, min_support = 0.6, use_colnames = True)  # itemsets에 제품명이 나오도록 지정  


,support,itemsets
0,0.6,(Apple)
1,0.8,(Beans)
2,0.6,(Cookie)
3,0.8,(Eggs)
4,0.6,(Milk)
5,0.6,(Yogurt)
6,0.6,"(Eggs, Apple)"
7,0.6,"(Cookie, Beans)"
8,0.6,"(Eggs, Beans)"
9,0.6,"(Beans, Yogurt)"


In [50]:
frequent_itemsets = apriori(df, min_support = 0.6, use_colnames = True)  
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x:len(x)) #length 추가  
frequent_itemsets

,support,itemsets,length
0,0.6,(Apple),1
1,0.8,(Beans),1
2,0.6,(Cookie),1
3,0.8,(Eggs),1
4,0.6,(Milk),1
5,0.6,(Yogurt),1
6,0.6,"(Eggs, Apple)",2
7,0.6,"(Cookie, Beans)",2
8,0.6,"(Eggs, Beans)",2
9,0.6,"(Beans, Yogurt)",2


In [51]:
frequent_itemsets[(frequent_itemsets['length'] == 2) & (frequent_itemsets['support'] >= 0.6)]  
#패턴 내 아이템 2개이고, 최소 지지도 60% 이상인 패턴만 필터링  

,support,itemsets,length
6,0.6,"(Eggs, Apple)",2
7,0.6,"(Cookie, Beans)",2
8,0.6,"(Eggs, Beans)",2
9,0.6,"(Beans, Yogurt)",2
10,0.6,"(Cookie, Eggs)",2


In [52]:
rules = association_rules (frequent_itemsets, metric = 'confidence', min_threshold = 0.7) 
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Eggs),(Apple),0.8,0.6,0.6,0.75,1.250000,0.12,1.6
1,(Apple),(Eggs),0.6,0.8,0.6,1.00,1.250000,0.12,inf
2,(Cookie),(Beans),0.6,0.8,0.6,1.00,1.250000,0.12,inf
3,(Beans),(Cookie),0.8,0.6,0.6,0.75,1.250000,0.12,1.6
4,(Eggs),(Beans),0.8,0.8,0.6,0.75,0.937500,-0.04,0.8
5,(Beans),(Eggs),0.8,0.8,0.6,0.75,0.937500,-0.04,0.8
6,(Beans),(Yogurt),0.8,0.6,0.6,0.75,1.250000,0.12,1.6
7,(Yogurt),(Beans),0.6,0.8,0.6,1.00,1.250000,0.12,inf
8,(Cookie),(Eggs),0.6,0.8,0.6,1.00,1.250000,0.12,inf
9,(Eggs),(Cookie),0.8,0.6,0.6,0.75,1.250000,0.12,1.6


In [53]:
# 향상도 1.2 이상인 패턴만 추출 # 향상도 min_threshold가 1이면 독립
rules2 = association_rules(frequent_itemsets, metric = 'lift', min_threshold = 1.2)  
rules2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Eggs),(Apple),0.8,0.6,0.6,0.75,1.250000,0.12,1.6
1,(Apple),(Eggs),0.6,0.8,0.6,1.00,1.250000,0.12,inf
2,(Cookie),(Beans),0.6,0.8,0.6,1.00,1.250000,0.12,inf
3,(Beans),(Cookie),0.8,0.6,0.6,0.75,1.250000,0.12,1.6
4,(Beans),(Yogurt),0.8,0.6,0.6,0.75,1.250000,0.12,1.6
5,(Yogurt),(Beans),0.6,0.8,0.6,1.00,1.250000,0.12,inf
6,(Cookie),(Eggs),0.6,0.8,0.6,1.00,1.250000,0.12,inf
7,(Eggs),(Cookie),0.8,0.6,0.6,0.75,1.250000,0.12,1.6
8,"(Cookie, Eggs)",(Beans),0.6,0.8,0.6,1.00,1.250000,0.12,inf
9,"(Cookie, Beans)",(Eggs),0.6,0.8,0.6,1.00,1.250000,0.12,inf


In [54]:
# 결과 중 필터링  ( 앞서구매하는 item 개수는 2개이상, 신뢰도 0.75이상, 향상도 1.2 이상)  

rules['antecedent_len'] = rules['antecedents'].apply(lambda x: len(x))  

In [55]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(Eggs),(Apple),0.8,0.6,0.6,0.75,1.250000,0.12,1.6,1
1,(Apple),(Eggs),0.6,0.8,0.6,1.00,1.250000,0.12,inf,1
2,(Cookie),(Beans),0.6,0.8,0.6,1.00,1.250000,0.12,inf,1
3,(Beans),(Cookie),0.8,0.6,0.6,0.75,1.250000,0.12,1.6,1
4,(Eggs),(Beans),0.8,0.8,0.6,0.75,0.937500,-0.04,0.8,1
5,(Beans),(Eggs),0.8,0.8,0.6,0.75,0.937500,-0.04,0.8,1
6,(Beans),(Yogurt),0.8,0.6,0.6,0.75,1.250000,0.12,1.6,1
7,(Yogurt),(Beans),0.6,0.8,0.6,1.00,1.250000,0.12,inf,1
8,(Cookie),(Eggs),0.6,0.8,0.6,1.00,1.250000,0.12,inf,1
9,(Eggs),(Cookie),0.8,0.6,0.6,0.75,1.250000,0.12,1.6,1


In [56]:
rules[(rules['antecedent_len'] >= 2) & (rules['confidence'] > 0.75) & (rules['lift'] > 1.2)]  

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
10,"(Cookie, Eggs)",(Beans),0.6,0.8,0.6,1.0,1.250000,0.12,inf,2
11,"(Cookie, Beans)",(Eggs),0.6,0.8,0.6,1.0,1.250000,0.12,inf,2
12,"(Beans, Eggs)",(Cookie),0.6,0.6,0.6,1.0,1.666667,0.24,inf,2
